## Задача 1
Для каждого дня в таблице `orders` рассчитайте следующие показатели:
> 1. Выручку, полученную в этот день.
> 2. Суммарную выручку на текущий день
> 3. Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.

```sql
SELECT date,
       revenue,
       sum(revenue) OVER (ORDER BY date) as total_revenue,
       round(100 * (revenue - lag(revenue, 1) OVER (ORDER BY date))::decimal / lag(revenue, 1) OVER (ORDER BY date),2) as revenue_change
FROM   (SELECT creation_time::date as date,
               sum(price) as revenue
        FROM   (SELECT creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using (product_id)
        GROUP BY date) t2 
```

## Задача 2
Для каждого дня в таблицах `orders` и `user_actions` рассчитайте следующие показатели:
> 1. Выручку на пользователя `(ARPU)` за текущий день.
> 2. Выручку на платящего пользователя `(ARPPU)` за текущий день.
> 3. Выручку с заказа, или средний чек `(AOV)` за текущий день.

```SQL
with 
not_cancel_orders as
    (SELECT 
        creation_time:: date as date,
        UNNEST(product_ids) as product_id,
        order_id
    FROM
        orders
    where
        order_id not in (SELECT order_id from user_actions where action = 'cancel_order')),
    
revenue_table as
    (SELECT
        date,
        sum(price) as revenue,
        COUNT(DISTINCT order_id) as paid_orders
    from
        not_cancel_orders
    join products
    ON not_cancel_orders.product_id = products.product_id
    GROUP BY
        date),

count_users as
    (SELECT
        time::date as date,
        COUNT(DISTINCT user_id) as all_users,
        COUNT(DISTINCT user_id) filter (where order_id not in (SELECT order_id from user_actions where action = 'cancel_order')) as paying_users
    from
        user_actions
    GROUP BY
        date)

SELECT
    revenue_table.date,
    round(revenue / all_users:: decimal, 2) as arpu,
    round(revenue / paying_users:: decimal, 2) as arppu,
    round(revenue / paid_orders:: decimal, 2) as aov
from
    revenue_table
join
    count_users using(date)
order by
    date
```

## Задача 3
По таблицам `orders` и `user_actions` для каждого дня рассчитайте следующие показатели:
> 1. Накопленную выручку на пользователя `(Running ARPU)`.
> 2. Накопленную выручку на платящего пользователя `(Running ARPPU)`.
> 3. Накопленную выручку с заказа, или средний чек `(Running AOV)`.

```sql
with 
not_cancel_orders as
    (SELECT 
        creation_time:: date as date,
        UNNEST(product_ids) as product_id,
        order_id
    FROM
        orders
    where
        order_id not in (SELECT order_id from user_actions where action = 'cancel_order')),
    
revenue_table as
    (SELECT
        date,
        sum(price) as revenue,
        COUNT(DISTINCT order_id) as paid_orders
    from
        not_cancel_orders
    join products
    ON not_cancel_orders.product_id = products.product_id
    GROUP BY
        date),

new_all_users_t as
    (SELECT
        date,
        count(user_id) as new_all_users
    from
        (
        SELECT
            user_id,
            min(time::date) as date
        from
            user_actions
        GROUP by user_id
        ) as t1
    GROUP by 
        date),

new_paying_users_t as 
    (SELECT
        date,
        count(user_id) as new_paying_users
    from
        (
        SELECT
            user_id,
            min(time::date) as date
        from
            user_actions
        where
            order_id not in (SELECT order_id from user_actions where action = 'cancel_order')
        GROUP by user_id
        ) as t2
    GROUP BY
        date)
    
SELECT
    revenue_table.date,
    round(sum(revenue) OVER(ORDER BY date) / sum(new_all_users) OVER (order by date):: decimal, 2) as running_arpu,
    round(sum(revenue) OVER(ORDER BY date) / sum(new_paying_users) OVER (order by date):: decimal, 2) as running_arppu,
    round(sum(revenue) OVER(ORDER BY date) / sum(paid_orders) OVER (order by date):: decimal, 2) as running_aov
from
    revenue_table
join
    new_all_users_t using(date)
join
    new_paying_users_t using(date)
order by
    date
```

## Задача 4
Посчитайте те же показатели, но в другом разрезе — не просто по дням, а по дням недели.
<br>Для каждого дня недели в таблицах `orders` и `user_actions` рассчитайте следующие показатели:
> 1. Выручку на пользователя `(ARPU)`.
> 2. Выручку на платящего пользователя `(ARPPU)`.
> 3. Выручку на заказ `(AOV)`.

При расчётах учитывайте данные только за период с 26 августа 2022 года по 8 сентября 2022 года включительно — так, чтобы в анализ попало одинаковое количество всех дней недели (ровно по два дня).

В результирующую таблицу включите как наименования дней недели (например, Monday), так и порядковый номер дня недели (от 1 до 7, где 1 — это Monday, 7 — это Sunday).

```sql
with 
not_cancel_orders as
    (SELECT 
        creation_time:: date as date,
        UNNEST(product_ids) as product_id,
        order_id
    FROM
        orders
    where
        order_id not in (SELECT order_id from user_actions where action = 'cancel_order')),
    
revenue_table as
    (SELECT
        TO_CHAR(date,'Day') as weekday,
        DATE_PART('isodow',date) as weekday_number,
        sum(price) as revenue,
        COUNT(DISTINCT order_id) as paid_orders
    from
        not_cancel_orders
    join products
    ON not_cancel_orders.product_id = products.product_id
    where
        date BETWEEN '2022-08-26' and '2022-09-08'
    GROUP BY
        weekday,
        weekday_number),

count_users as
    (SELECT
        TO_CHAR(time::date,'Day') as weekday,
        DATE_PART('isodow',time::date) as weekday_number,
        COUNT(DISTINCT user_id) as all_users,
        COUNT(DISTINCT user_id) filter (where order_id not in (SELECT order_id from user_actions where action = 'cancel_order')) as paying_users
    from
        user_actions
    where
        time::date BETWEEN '2022-08-26' and '2022-09-08'
    GROUP BY
        weekday,
        weekday_number)

SELECT
    count_users.weekday,
    count_users.weekday_number,
    round(revenue / all_users:: decimal, 2) as arpu,
    round(revenue / paying_users:: decimal, 2) as arppu,
    round(revenue / paid_orders:: decimal, 2) as aov
from
    revenue_table
join
    count_users using(weekday)
order by
    count_users.weekday_number
```

## Задача 5
Немного усложним наш первоначальный запрос и отдельно посчитаем ежедневную выручку с заказов новых пользователей нашего сервиса. Посмотрим, какую долю она составляет в общей выручке с заказов всех пользователей — и новых, и старых.
<br>Для каждого дня в таблицах `orders` и `user_actions` рассчитайте следующие показатели:
> 1. Выручку, полученную в этот день.
> 2. Выручку с заказов новых пользователей, полученную в этот день.
> 3. Долю выручки с заказов новых пользователей в общей выручке, полученной за этот день.
> 4. Долю выручки с заказов остальных пользователей в общей выручке, полученной за этот день.

```sql
with 
    t1 as
        (
        SELECT
            creation_time::date as date,
            order_id,
            UNNEST(product_ids) as product_id
        FROM
            orders
        WHERE
            order_id not in (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
        ),
    t2 as
        (
        SELECT
            date,
            order_id,
            product_id,
            price,
            user_id
        FROM
            t1
        join
            products using(product_id)
        join
            user_actions using(order_id)
        ),
    revenue as
        (
        SELECT
            date,
            sum(price) as revenue
        FROM
            t2
        GROUP BY
            date
        ),
    new_users as
        (
        SELECT
            min(time::date) as min_date,
            user_id
        FROM
            user_actions
        group by
            user_id
        ),
    new_users_revenue as
        (
        SELECT
            date,
            sum(price) as new_users_revenue
        FROM
            t2
        right join
            new_users
        ON  t2.date = new_users.min_date and t2.user_id = new_users.user_id
        GROUP by
            date
        )


SELECT
    date,
    revenue,
    new_users_revenue,
    ROUND (100*new_users_revenue/revenue::DECIMAL,2) AS new_users_revenue_share,
    (100 - ROUND (100*new_users_revenue/revenue::DECIMAL,2)) AS old_users_revenue_share
FROM
    revenue
join
    new_users_revenue using(date)
order by
    date
```

## Задача 6
Также было бы интересно посмотреть, какие товары пользуются наибольшим спросом и приносят нам основной доход.
Для каждого товара, представленного в таблице `products`, за весь период времени в таблице `orders` рассчитайте следующие показатели:
> 1. Суммарную выручку, полученную от продажи этого товара за весь период.
> 2. Долю выручки от продажи этого товара в общей выручке, полученной за весь период.

Товары, округлённая доля которых в выручке составляет менее 0.5%, объедините в общую группу с названием «ДРУГОЕ» (без кавычек), просуммировав округлённые доли этих товаров.

```sql
with
    main_table as
        (
        SELECT
            creation_time,order_id,name,price,product_id
        FROM
            (SELECT UNNEST(product_ids) as product_id,* FROM orders) as t1
        join products using(product_id)
        where
            order_id not in (SELECT order_id FROM user_actions where action = 'cancel_order')
        ),
    revenue_from_the_product as
        (
        SELECT
            product_id,
            SUM(price) as revenue_from_the_product
        FROM
            main_table
        group by
            product_id
        ),
    final_table as
        (
        SELECT
            product_id,
            revenue_from_the_product,
            round(100*(revenue_from_the_product::decimal / SUM(revenue_from_the_product) OVER()),2) as share_in_revenue
        FROM
            revenue_from_the_product
        ORDER BY 
            share_in_revenue desc
        )
        
SELECT
    case
        when share_in_revenue < 0.5 then 'ДРУГОЕ'
        else name
    end as product_name,
    sum(revenue_from_the_product) as revenue,
    sum(share_in_revenue) as share_in_revenue
FROM
    final_table
join
    products using(product_id)
GROUP BY
    product_name
order by
    revenue desc

```

## Задача 7
Теперь попробуем учесть в наших расчётах затраты с налогами и посчитаем валовую прибыль, то есть ту сумму, которую мы фактически получили в результате реализации товаров за рассматриваемый период.

Для каждого дня в таблицах `orders` и `courier_actions` рассчитайте следующие показатели:

1. Выручку, полученную в этот день.
2. Затраты, образовавшиеся в этот день.
3. Сумму НДС с продажи товаров в этот день.
4. Валовую прибыль в этот день (выручка за вычетом затрат и НДС).
5. Суммарную выручку на текущий день.
6. Суммарные затраты на текущий день.
7. Суммарный НДС на текущий день.
8. Суммарную валовую прибыль на текущий день.
9. Долю валовой прибыли в выручке за этот день (долю п.4 в п.1).
10. Долю суммарной валовой прибыли в суммарной выручке на текущий день (долю п.8 в п.5).

Чтобы посчитать затраты, в этой задаче введём дополнительные условия.

В упрощённом виде затраты нашего сервиса будем считать как сумму постоянных и переменных издержек. К постоянным издержкам отнесём аренду складских помещений, а к переменным — стоимость сборки и доставки заказа. Таким образом, переменные затраты будут напрямую зависеть от числа заказов.

Из данных, которые нам предоставил финансовый отдел, известно, что в августе 2022 года постоянные затраты составляли 120 000 рублей в день. Однако уже в сентябре нашему сервису потребовались дополнительные помещения, и поэтому постоянные затраты возросли до 150 000 рублей в день.

Также известно, что в августе 2022 года сборка одного заказа обходилась нам в 140 рублей, при этом курьерам мы платили по 150 рублей за один доставленный заказ и ещё 400 рублей ежедневно в качестве бонуса, если курьер доставлял не менее 5 заказов в день. В сентябре продакт-менеджерам удалось снизить затраты на сборку заказа до 115 рублей, но при этом пришлось повысить бонусную выплату за доставку 5 и более заказов до 500 рублей, чтобы обеспечить более конкурентоспособные условия труда. При этом в сентябре выплата курьерам за один доставленный заказ осталась неизменной.

При расчёте переменных затрат учитывайте следующие условия:

1. Затраты на сборку учитываются в том же дне, когда был оформлен заказ. Сборка отменённых заказов не производится.

2. Выплата курьерам за доставленный заказ начисляется сразу же после его доставки, поэтому если курьер доставит заказ на следующий день, то и выплата будет учтена в следующем дне.

3. Для получения бонусной выплаты курьерам необходимо доставить не менее 5 заказов в течение одного дня, поэтому если курьер примет 5 заказов в течение дня, но последний из них доставит после полуночи, бонусную выплату он не получит.

При расчёте НДС учитывайте, что для некоторых товаров налог составляет 10%, а не 20%.

```sql
with 
    costs as
        (
        SELECT
            date,
            case
                when date < '2022-09-01' then 120000 + SUM(costs_for_orders) + SUM(bonus)
                else 150000 + SUM(costs_for_orders) + SUM(bonus)
            end as costs
        FROM
            (SELECT
                date, courier_id,
                case
                    when date < '2022-09-01' and deliver_orders >=5 then 400
                    when date >= '2022-09-01' and deliver_orders >=5 then 500
                    else 0
                end as bonus,
                case
                    when date < '2022-09-01' then accept_orders * 140 + deliver_orders * 150
                    else accept_orders * 115 + deliver_orders * 150
                end as costs_for_orders
            FROM
                (SELECT
                    time::date as date, courier_id,
                    COUNT(order_id) FILTER(WHERE action = 'deliver_order') as deliver_orders,
                    COUNT(order_id) FILTER(WHERE action = 'accept_order') as accept_orders
                FROM
                    courier_actions
                WHERE
                    order_id not in (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
                group by
                    date, courier_id) as t1) as t2
        GROUP BY
            date
        order by
            date
        ),
    revenue_and_tax as
        (
        SELECT
            date,
            SUM(price) as revenue,
            SUM(vat) as tax
        FROM    
            (SELECT
                date,order_id,price,name,product_id,
                case
                    when name in ('сахар', 'сухарики', 'сушки', 'семечки', 'масло льняное', 'виноград', 'масло оливковое', 'арбуз', 'батон', 'йогурт', 'сливки', 'гречка', 'овсянка', 'макароны', 'баранина', 'апельсины', 'бублики', 'хлеб', 'горох', 'сметана', 'рыба копченая', 'мука', 'шпроты', 'сосиски', 'свинина', 'рис', 'масло кунжутное', 'сгущенка', 'ананас', 'говядина', 'соль', 'рыба вяленая', 'масло подсолнечное', 'яблоки', 'груши', 'лепешка', 'молоко', 'курица', 'лаваш', 'вафли', 'мандарины')
                    then round(price*10/110,2)
                    else round(price*20/120,2)
                end as vat
            FROM
                (SELECT creation_time::date as date, order_id, UNNEST(product_ids) as product_id FROM orders) as t1
            join
                products using(product_id)
            WHERE
                order_id not in (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) as t2
        GROUP BY
            date
        )

SELECT
    date, revenue, costs,tax,
    revenue - costs - tax as gross_profit,
    SUM(revenue) OVER(ORDER BY date) as total_revenue,
    SUM(costs) OVER(ORDER BY date) as total_costs,
    SUM(tax) OVER(ORDER BY date) as total_tax,
    SUM(revenue - costs - tax) OVER(ORDER BY date) as total_gross_profit,
    round(100 * (revenue - costs - tax) / revenue::decimal,2) as gross_profit_ratio,
    round(100 * (SUM(revenue - costs - tax) OVER(ORDER BY date)) / SUM(revenue) OVER(ORDER BY date)::decimal,2) as total_gross_profit_ratio
FROM
    revenue_and_tax
join
    costs using(date)
```